In [2]:
import random 
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
transition_matrix = pd.read_csv('./data/transition_matrices/transition_matrix_average.csv', index_col=0)
transition_matrix

,checkout,dairy,drinks,fruit,spices
location,,,,,
checkout,1.000000,0.000000,0.000000,0.000000,0.000000
dairy,0.103427,0.736750,0.058597,0.049834,0.051392
drinks,0.216188,0.010903,0.597958,0.087933,0.087018
fruit,0.201977,0.096013,0.054791,0.596498,0.050722
spices,0.150493,0.193610,0.163121,0.090887,0.401889


In [4]:
class Customer:
    """
    a single customer that moves through the supermarket
    in a MCMC simulation
    """
    def __init__(self, name, state, budget=100):
        self.name = name
        self.state = state
        self.budget = budget
    def __repr__(self):
        return f'customer_no {self.name} {self.state}'
    def next_state(self):
        '''
        Propagates the customer to the next state.
        Returns nothing.
        '''
        self.state = random.choices(['checkout', 'dairy', 'drinks', 'fruit', 'spices'], list(transition_matrix.loc[self.state]) )
        self.state = self.state[0]
        
        
    def is_active(self):
        
        if self.state == 'checkout':
            return True
        

In [5]:
class Supermarket:
    """manages multiple Customer instances that are currently in the market."""

    def __init__(self,market_name,opening,closing): 
        self.market_name = market_name
        self.opening = opening
        self.closing = closing
        self.customers = []
        self.current_time = 0
        self.index=0
        self.customer_index =0
        self.state = None
        self.dti = pd.date_range(self.opening, self.closing, freq="T").time
        
        
    def __repr__(self):
        return f'{sefl.market_name}'
    
    def is_open(self):
        if self.index <= len(self.dti)-2:
            return datetime.strptime(self.opening, '%H:%M:%S') <= datetime.strptime(self.get_time(), '%H:%M:%S') <= datetime.strptime(self.closing, '%H:%M:%S')

    def get_time(self):
        """current time in HH:MM format,"""
        self.current_time = str(self.dti[self.index])
        return self.current_time
        
    def print_customers(self):
        """print all customers with the current time and id in CSV format.
        """
        return self.customers

    def next_minute(self):
        """"propagates all customers to the next state."""
        
        self.index += 1
        next_time = self.dti[self.index] 
        
        for customer in self.customers:
            customer.next_state()

            
    def add_new_customers(self):
        """randomly creates new customers.
        """
        self.state = random.choices(['dairy', 'drinks', 'fruit', 'spices'])
        self.state = self.state[0]
        self.customer_index +=1
        new_customer = Customer(self.customer_index,self.state)
        self.customers.append(new_customer)
        

    def remove_exitsting_customers(self):
        """removes every customer that is not active any more.
        """
        for customer in self.customers:
            if customer.is_active():
                self.customers.remove(customer)

In [6]:
#initialization for the while-loop
a = Supermarket('MarketRangers','07:00:00','22:00:00')
market_rangers = pd.DataFrame()

while a.is_open():    
    a.add_new_customers()
    for customer in a.customers:
        market_rangers = market_rangers.append({'time': a.get_time(), 
                                                'customer_no': str(customer.name),
                                                'location': customer.state
                                               },ignore_index=True)
    
    a.remove_exitsting_customers()
    a.next_minute()
    
market_rangers.set_index('time', inplace=True)
market_rangers.head()   

,customer_no,location
time,,
07:00:00,1,dairy
07:01:00,1,dairy
07:01:00,2,drinks
07:02:00,1,dairy
07:02:00,2,drinks


In [7]:
market_rangers.to_csv('./data/Simulated_Market_Table/simulated_market_table_average.csv')